# Download SWOT swaths that intersect with a specific region

### Here we'll use some scripts to download SWOT data in a specific region. 

- This notebook uses scripts from <code>tatsu_src/tatsu_download_swaths.py</code>, and is partially based on code taken from the [SWOT-OpenToolkit](https://github.com/SWOT-community/SWOT-OpenToolkit) developed by Jinbo Wang (Texas A&M).
- Note that we are using <code>paramiko</code> to directly pull swaths from the [AVISO](https://www.aviso.altimetry.fr/en/home.html) altimetry database using <code>sftp</code>. Limiting ourselves to sftp is a little inefficient, especially if we only want subsets of particular region, but it's more straightforward than fighting PO.DAAC and gives us access to the most up-to-date datasets.
- You can just use my login information, provided here, to access AVISO
- NOTE: As an example, I wrote this script to save L3 SWOT swaths to a folder in the base directory named "<code>SWOT_L3</code>", which has been added to the <code>.gitignore</code> file. You should change the save path to wherever is most convienent for you. If you save your files to a different directory on the main path remember to add the save directory to <code>.gitignore</code> to avoid problems with large files.


In [1]:
"""
Download SWOT swaths in a specific bounding box.

Dependencies:
paramiko
numpy
xarray

Author: Tatsu Monkman
Date: 2025-01-24
"""
import os
import time

# Add the path to the Tatsu's swot download library
import sys
sys.path.append('../src/')
import tatsu_download_swaths as tatsu_download


In [2]:
os.getcwd()

'/scratch/projects/shaferlab/tatsu/NYU_SWOT_project/SWOT-data-analysis/examples'

## Specify AVISO login info to access AVISO via sftp


In [3]:
# Define sftp connection parameters. Here I am just using my own login info. 
# Feel free to change, but you should probable leave the "port" the same
ssh_kwargs = {
              "hostname":"ftp-access.aviso.altimetry.fr",
              "port": 2221,
              "username":"tdmonkman@uchicago.edu",
              "password":"2prSvl"
            }


## Define lat/lon bounds for the region you are interested in

In [4]:
# Define bounding domain, here I'm downloading some swaths 
# over the Kuroshio east of Japan.

# Rough East of Japan domain (Kuroshio-ish)
kuroshio_sw_corner = [140,15]
kuroshio_ne_corner = [170,40]

# NOTE: I am specifically recording the Northern and Southern edges
# of the bounding box here for later use when I subset the data.
# I find it helpful to add them to a new list to keep everything 
# organized.
kuroshio_lat_lims = [kuroshio_sw_corner[1],kuroshio_ne_corner[1]]

## Specify the cycles you want to download

Key points:
- SWOT data is organized by orbit "cycle" on AVISO. Each numbered cycle corresponds to one full traversal of the individual swaths that make up a complete orbital cycle. Cycles numbered 474-578 correspond to 1-day repeat orbits made during the Cal/Val phase, while cycles numbered 001-016+ correspond to orbits made during the science phase. Note you need to keep the leading zeros on the labels for the low-numbered cycles since the filenames are use standardized lengths..
- Each cycle is subdivided into individual "passes" which correspond to one North-South or South-North swath of satellite data. Each pass is given an idenitication number, which can be used to find specific swaths you are interested in. For example, the orbital shape files in  <code>../orbit_data/</code> divide the orbits by "PASS_ID", which is just the pass number of each swath.
- Pass IDs correspond to the order in which each pass occured during each cycle. For example, "Pass 1" in the science phase is the first pass of a given science cycle, and "Pass 1" in the Cal/Val phase is the first pass in a given Cal/Val cycle.
- Every "pass" is repeated for every "cycle". For example, if you want to find all the SWOT data in the California region you would first look at which passes travel through the region, then download those specific passes for all of the cycles you are interested in.

In [5]:
"""
Below is some code to specify the passes and cycles to download.

Here we are downloading passes in the Kuroshio from all of the Cal/Val cycles,
i.e. cycles 474-578. You will note that I've included (commented out) code for 
pulling the science cycles (cycles 001-016). 

Next, I specify which samples I want using the "tatsu_download.find_swaths()" script
located in ../src/. This script basically just does what is already in the 001_....ipynb 
example notebook. Note that you need to include the 

"""

# EXAMPLE: Cycles 001 - 016 are for the 21-day science phase
# cycles = [str(c_num).zfill(3) for c_num in range(1,17)]

# EXAMPLE: Cycles 474 - 578 are from the 1-day repeat Cal/Val phase
# cycles = [str(c_num).zfill(3) for c_num in range(474,578)]

# Here I'm just pulling the first 10 cycles from the 1-day repeat Cal/Val phase
cycles = [str(c_num).zfill(3) for c_num in range(474,484)]


# Use sph_science_swath for the 21-day repeat
# path_to_sph_file="../../orbit_data/sph_science_swath.zip"
# Use sph_calval_swath for the 1-day repeat
path_to_sph_file="../../orbit_data/sph_calval_swath.zip"

# Get pass IDs for swaths that intersect your box
pass_IDs_list = tatsu_download.find_swaths(kuroshio_sw_corner, kuroshio_ne_corner,
                                           path_to_sph_file=path_to_sph_file)



/ext3/miniforge3/envs/swot_python12/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: /vsizip/../../orbit_data/sph_calval_swath.zip/swot_calval_orbit_june2015-v2_swath.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


<code>pass_IDs_list</code> is just a string of the pass IDS for the passes that travel through the kuroshio

e.g.

In [6]:
pass_IDs_list

['004', '019']

## Specify the path to the data on the AVISO ftp server

Next we want to specify where to look for the data on the AVISO ftp server. I'm looking for version 1.0.2 release of the SWOT L3 data, so I've specified the path to this specific release on the AVISO server:

```python
    # Paths for L3 unsmoothed data
    remote_path="swot_products/l3_karin_nadir/l3_lr_ssh/v1_0_2/Unsmoothed"
```

If you want to poke around on the AVISO ftp server to find different data releases, you'll have to logon to the AVISO ftp server using your AVISO username / password (or mine if you don't want to make your own), for example:

```unix
>>> sftp -o User=tdmonkman@uchicago.edu  sftp://ftp-access.aviso.altimetry.fr:2221/swot_products/
    tdmonkman@uchicago.edu@ftp-access.aviso.altimetry.fr's password: PASSWORD
    Connected to ftp-access.aviso.altimetry.fr.
    Changing to: /swot_products/

sftp> ls
    l2_karin        l3_karin_nadir  l4_karin_nadir 
```
Then you can go look around for the data you want using standard unix commands.

## Specify the remote path to access the data and the local_path you want to save it to on your machine
I'm saving the data in this example to the main SWOT-data-analysis directory, but you'll want to put it somewhere more permanent on your own machine. NOTE: If you want to add more data to SWOT-data-analysis don't forget to change the <code>.gitignore</code> file!

In [7]:
###########################################################################
# L3 v1.0.2 Unsmoothed data:
# Paths for L3 unsmoothed data in the kuroshio region
remote_path="swot_products/l3_karin_nadir/l3_lr_ssh/v1_0_2/Unsmoothed"
local_path = f"../SWOT_L3/Unsmoothed_kuroshio"


###########################################################################
# Some more example datasets:

# Paths for L3 expert data
#remote_path="/swot_products/l3_karin_nadir/l3_lr_ssh/v1_0_2/Expert"
#local_path = "../../../SWOT_L3/Expert"
# Paths for L2 expert data
#remote_path="/swot_products/l2_karin/l2_lr_ssh/PGC0/Expert"
#local_path = "../../../SWOT_L2/PGC0/Expert"




# Download the data

The script below downloads the data for the given passes and cycles you are interested in. Note I've instructed the script to generate an extra log file (the <code>skipped_swaths.txt</code> file) to record when there's a problem and the script fails to download the data.

## Using tatsu_download.download_passes()

I wrote a helper script to download the data using sftp via paramiko and to deal with subsetting the data. Hopefully it works.
```python
    tatsu_download.download_passes(pass_ID,cycle=cycle,remote_path=remote_path,
                                   save_path=save_path,**ssh_kwargs,
                                   subset=True,lat_lims=lat_lims,trim_suffix=f"kuroshio")
```
### Key points
- This script downloads each pass for each cycle separately in order to not piss off the AVISO admins (i.e. if you try doing this in parallel it gets flagged as suspicious by AVISO and they lock you out until you apologize. Apparently 10 SSH connections at a time is the limit, but I would just stick with downloading one pass at a time / per SSH connection in case of bugs).
- <code>subset=True</code> tells the script to trim the swaths by latitude, i.e. only downloading the latitudes that fall into your bounding box. This is nice since the swaths can sometimes get quite large. <code>subset=False</code> tells the script to just download the whole swath, which is fun. <code>lat_lims</code> and <code>trim_suffix</code> are only envoked if <code>subset=True</code>/
- <code>lat_lims</code> are the latitudes at which the script cuts off the swath. Should be input as <code>[southern_lat,northern_lat]</code>.
- <code>trim_suffix=f"kuroshio"</code> is a string that I add to the trimmed swath <code>.netcdf</code> files as a suffix to tell me that they're trimmed and to specify what region they correspond to. I find this useful for when things get complicated.



Good luck!

In [ ]:
# Make a file to store IDs for swaths that didn't download
with open("skipped_swaths.txt","w") as file:
    file.write("Failed to download the following swaths:")
    file.write(" cycle, pass_ID \n ----------")
    file.close()

# MAKE SURE TO CHANGE THE SAVE PATH IF YOU ARE REDOWNLOADING
for cycle in cycles:
    save_path = local_path+f"/cycle_{cycle}"
    if not os.path.isdir(save_path):
        os.makedirs(save_path, exist_ok=False)
    # If you want to clean any bad existing files in the path,
    # do so here. You need to modify the "size" parameter to specify
    # a size cutoff for what is an "incomplete" file. Generally a skipped 
    # connection will generate a filler file that is size < 10 Mb
    tatsu_download.clean_incomplete_files(save_path, size=10)
    
    # Download passes
    for pass_ID in pass_IDs_list:
        try:
            # Call the download passes script. Note the login info goes in "ssh_kwargs"
            tatsu_download.download_passes(pass_ID,cycle=cycle,remote_path=remote_path,
                                           save_path=save_path,**ssh_kwargs,
                                           subset=True,lat_lims=lat_lims,trim_suffix=f"kuroshio")
        except Exception as e:
            print("*****"*5)
            print(f"Could not download pass {pass_ID} in cycle {cycle}")
            print(f"An error occured: {e}")
            print("*****"*5)
            with open("skipped_swaths.txt","a") as file:
                file.write(f"\n {cycle}, {pass_ID}")
                
    # Sleep for 10 seconds so you don't make AVISO mad 
    time.sleep(10)


Attempting SSH connection...
Found cycle_474 on remote server
Looking for matches to SWOT_L3_LR_SSH_Unsmoothed_474_002...
Attempting SSH connection...
Found cycle_474 on remote server
Looking for matches to SWOT_L3_LR_SSH_Unsmoothed_474_004...
Attempting SSH connection...
Found cycle_474 on remote server
Looking for matches to SWOT_L3_LR_SSH_Unsmoothed_474_006...
Found remote file SWOT_L3_LR_SSH_Unsmoothed_474_006_20230329T021734_20230329T030839_v1.0.2.nc
Subsetting SWOT_L3_LR_SSH_Unsmoothed_474_006_20230329T021734_20230329T030839_v1.0.2.nc


/state/partition1/job-56258829/ipykernel_3365380/3414969896.py:22: SerializationWarning: saving variable latitude with floating point data as an integer dtype without any _FillValue to use for NaNs
  tatsu_download.download_passes(pass_ID,cycle=cycle,remote_path=remote_path,
/state/partition1/job-56258829/ipykernel_3365380/3414969896.py:22: SerializationWarning: saving variable longitude with floating point data as an integer dtype without any _FillValue to use for NaNs
  tatsu_download.download_passes(pass_ID,cycle=cycle,remote_path=remote_path,


Attempting SSH connection...
Found cycle_474 on remote server
Looking for matches to SWOT_L3_LR_SSH_Unsmoothed_474_017...
Found remote file SWOT_L3_LR_SSH_Unsmoothed_474_017_20230329T113936_20230329T123042_v1.0.2.nc
Subsetting SWOT_L3_LR_SSH_Unsmoothed_474_017_20230329T113936_20230329T123042_v1.0.2.nc


/state/partition1/job-56258829/ipykernel_3365380/3414969896.py:22: SerializationWarning: saving variable latitude with floating point data as an integer dtype without any _FillValue to use for NaNs
  tatsu_download.download_passes(pass_ID,cycle=cycle,remote_path=remote_path,
/state/partition1/job-56258829/ipykernel_3365380/3414969896.py:22: SerializationWarning: saving variable longitude with floating point data as an integer dtype without any _FillValue to use for NaNs
  tatsu_download.download_passes(pass_ID,cycle=cycle,remote_path=remote_path,


Attempting SSH connection...
Found cycle_474 on remote server
Looking for matches to SWOT_L3_LR_SSH_Unsmoothed_474_019...
Found remote file SWOT_L3_LR_SSH_Unsmoothed_474_019_20230329T132147_20230329T141138_v1.0.2.nc
Subsetting SWOT_L3_LR_SSH_Unsmoothed_474_019_20230329T132147_20230329T141138_v1.0.2.nc


...

/state/partition1/job-56258829/ipykernel_3365380/3414969896.py:22: SerializationWarning: saving variable latitude with floating point data as an integer dtype without any _FillValue to use for NaNs
  tatsu_download.download_passes(pass_ID,cycle=cycle,remote_path=remote_path,
/state/partition1/job-56258829/ipykernel_3365380/3414969896.py:22: SerializationWarning: saving variable longitude with floating point data as an integer dtype without any _FillValue to use for NaNs
  tatsu_download.download_passes(pass_ID,cycle=cycle,remote_path=remote_path,


Attempting SSH connection...
Found cycle_475 on remote server
Looking for matches to SWOT_L3_LR_SSH_Unsmoothed_475_002...
Found remote file SWOT_L3_LR_SSH_Unsmoothed_475_002_20230329T224349_20230329T233455_v1.0.2.nc
Subsetting SWOT_L3_LR_SSH_Unsmoothed_475_002_20230329T224349_20230329T233455_v1.0.2.nc


/state/partition1/job-56258829/ipykernel_3365380/3414969896.py:22: SerializationWarning: saving variable latitude with floating point data as an integer dtype without any _FillValue to use for NaNs
  tatsu_download.download_passes(pass_ID,cycle=cycle,remote_path=remote_path,
/state/partition1/job-56258829/ipykernel_3365380/3414969896.py:22: SerializationWarning: saving variable longitude with floating point data as an integer dtype without any _FillValue to use for NaNs
  tatsu_download.download_passes(pass_ID,cycle=cycle,remote_path=remote_path,


Attempting SSH connection...
Found cycle_475 on remote server
Looking for matches to SWOT_L3_LR_SSH_Unsmoothed_475_004...
Found remote file SWOT_L3_LR_SSH_Unsmoothed_475_004_20230330T002601_20230330T011706_v1.0.2.nc
Subsetting SWOT_L3_LR_SSH_Unsmoothed_475_004_20230330T002601_20230330T011706_v1.0.2.nc


/state/partition1/job-56258829/ipykernel_3365380/3414969896.py:22: SerializationWarning: saving variable latitude with floating point data as an integer dtype without any _FillValue to use for NaNs
  tatsu_download.download_passes(pass_ID,cycle=cycle,remote_path=remote_path,
/state/partition1/job-56258829/ipykernel_3365380/3414969896.py:22: SerializationWarning: saving variable longitude with floating point data as an integer dtype without any _FillValue to use for NaNs
  tatsu_download.download_passes(pass_ID,cycle=cycle,remote_path=remote_path,
